In [2]:
from sklearn.cluster import AgglomerativeClustering
from collections import defaultdict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos_sim
%run TFIDF_Summary.ipynb


class Cluster_process(Tfidf_process) :
    
    def summarize(self, corpus) :
        X = self.vectorizer.transform(corpus).toarray()
        C = AgglomerativeClustering(n_clusters = 3)
        Z = C.fit_predict(X)
        clu = defaultdict(list)
        for ind, cls in enumerate(Z) :
            clu[cls].append(ind)
        summ = []
        selected = set()
        total_length = 0
        while total_length < self.nb_words:
            for clus in clu.values() :
                clu_mat = []
                clu_ind = []
                for ind in clus :
                    if ind not in selected:
                        clu_mat.append(X[ind])
                        clu_ind.append(ind)
                if len(clu_ind) == 0:
                    continue
                centroid = np.sum(clu_mat, axis = 0)/len(clu_mat)
                best = np.argmax(cos_sim(clu_mat, centroid.reshape(1, -1)))
                summ.append(clu_ind[best])
                total_length += len(corpus[clu_ind[best]].split())
                if total_length > 100:
                    return sorted(summ)
                selected.add(clu_ind[best])
        return sorted(summ)